# Deep_CNN_MNIST
This code below is all about CNN model to classify MNIST datasets. It is demonstrating a deep CNN model code simulation.

### Header

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init as init
from torch.utils.data import DataLoader

### Select between 'cuda' or 'cpu'

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device: {}'.format(device))

device: cuda


### Give a random seed

In [18]:
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

### Hyper parameters

In [19]:
batch_size = 100
learning_rate = 1e-5
training_epochs = 15

### Dataset

In [20]:
MNIST_train = dsets.MNIST(root='MNIST_data/', train=True,  transform=transforms.ToTensor(), download=True)
MNIST_test  = dsets.MNIST(root='MNIST_data/', train=False, transform=transforms.ToTensor(), download=True)

dataloader = DataLoader(dataset=MNIST_train, batch_size=batch_size, shuffle=True, drop_last=True)

In [21]:
iteration = len(dataloader)
print('total batch(iteration): {}'.format(iteration))

total batch(iteration): 600


In [22]:
print('input data size: ',MNIST_train.train_data.shape)
print('input label size: ',MNIST_train.train_labels.shape)

input data size:  torch.Size([60000, 28, 28])
input label size:  torch.Size([60000])


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:62: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:52: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


### Model

In [23]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.keep_prob=0.5
        
        # image size = (?(60000), 1,28,28 )
        # Conv2d    => (?(60000),32,28,28 )
        # Pool      => (?(60000),32,14,14 )    
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # image size = (?(60000),32,14,14 )
        # Conv2d    => (?(60000),64,14,14 )
        # Pool      => (?(60000),64,7,7 )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # image size = (?(60000),64,7,7 )
        # Conv2d    => (?(60000),128,7,7 )
        # Pool      => (?(60000),128,4,4 )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2,padding=1)
        )
        
        self.fc1 = nn.Linear(4*4*128, 625, bias = True)
        self.fc2 = nn.Linear(625, 10, bias=True)
        init.xavier_uniform_(self.fc1.weight)
        init.xavier_uniform_(self.fc2.weight)

        self.layer4 = nn.Sequential(
            self.fc1,
            nn.ReLU(),
            torch.nn.Dropout(p=1-self.keep_prob)
        )

        self.layer5 = nn.Sequential(
            self.fc2,
            nn.ReLU(),
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.layer4(out)
        out = self.layer5(out)
        return out

model = CNN().to(device)

### Optimzer & Cost function

In [24]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss().to(device)

### Training

In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0
    for (X,Y) in dataloader:
        X = X.to(device)
        Y = Y.to(device)
        
        hypothesis = model(X)
        cost = loss_fn(hypothesis, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost / iteration
    print('Epoch: {:4d}/{}, Cost: {:.5f}'.format(epoch, training_epochs, avg_cost))

Epoch:    0/15, Cost: 1.85063


### Validation

In [ ]:
with torch.no_grad():
    print(MNIST_test.test_data.size())
    X_test = MNIST_test.test_data.view(len(MNIST_test),1,28,28).float().to(device)
    print(MNIST_test.test_data.view(len(MNIST_test),1,28,28).size())
    Y_test = MNIST_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, dim=1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: {:.7f}'.format(accuracy.item()))    